In [3]:
!pip install pandas

import pandas as pd

# Load the EEG dataset
eeg_data = pd.read_csv('eeg.csv')




In [4]:
# Check the first few rows to understand the structure
print(eeg_data.head())


            time         P3         C3        F3         Fz         F4  \
0  183779.930867 -23.763599 -31.924854 -1.934982 -79.180916 -86.278717   
1  183779.930913 -26.475424 -31.927679 -6.166468 -81.603569 -92.313293   
2  183779.930932 -20.439611 -26.794561 -1.640890 -79.197510 -86.579071   
3  183779.930950 -27.830748 -31.778809 -2.383846 -79.627190 -89.444626   
4  183779.930966 -29.192734 -33.141907 -1.619983 -77.948120 -85.371559   

          C4         P4         Cz          Pz  ...          O2      X3  \
0 -67.345665 -83.710152 -58.295597 -133.929596  ... -115.251083 -12.642   
1 -72.470337 -82.810837 -60.407738 -133.932419  ... -117.061508 -12.341   
2 -65.229378 -76.173592 -57.987892 -127.896614  ... -109.820625 -12.943   
3 -67.497818 -82.062004 -58.447430 -136.190750  ... -119.922363 -12.341   
4 -66.145111 -84.629982 -60.709770 -140.863739  ... -129.716873 -12.642   

          X2          F7          F8     X1         A2          T6  \
0 -20.167000 -710.807129 -150.4972

In [6]:
!pip install scipy



   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/44.5 MB ? eta -:--:--
   - -------------------------------------- 1.3/44.5 MB 4.0 MB/s eta 0:00:11
   - -------------------------------------- 1.6/44.5 MB 4.7 MB/s eta 0:00:10
   -- ------------------------------------- 2.9/44.5 MB 4.3 MB/s eta 0:00:10
   --- ------------------------------------ 3.7/44.5 MB 3.8 MB/s eta 0:00:11
   --- ------------------------------------ 3.9/44.5 MB 3.6 MB/s eta 0:00:12
   ---- ----------------------------------- 4.7/44.5 MB 3.4 MB/s eta 0:00:12
   ---- ----------------------------------- 5.0/44.5 MB 3.1 MB/s eta 0:00:13
   ---- ----------------------------------- 5.5/44.5 MB 3.0 MB/s eta 0:00:14
   ----- ---------------------------------- 6.3/44.5 MB 3.1 MB/s eta 0:00:13
   ----- ---------------------------------- 6.6/44.5 MB 2.9 MB/s eta 0:00:14
   ------ --------------------------------- 7.1/44.5 MB 2.9 MB/s eta 0:00:13
   ------ ---

In [7]:
from scipy.signal import butter, filtfilt

# Define a function to apply a band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_data = filtfilt(b, a, data, axis=0)
    return filtered_data

# Assuming the dataset contains EEG channels, let's filter them (assuming a sample rate of 250 Hz)
fs = 250  # Sampling rate (Hz)
lowcut = 7  # Lower bound of SSVEP frequency
highcut = 70  # Upper bound of SSVEP frequency

# Apply the bandpass filter to each channel of EEG data
filtered_eeg = eeg_data.apply(lambda x: bandpass_filter(x, lowcut, highcut, fs))
print(filtered_eeg.head())


       time        P3        C3        F3        Fz        F4        C4  \
0 -0.001096  0.014405 -0.037070 -0.113205 -0.173458 -0.217919  0.038756   
1 -0.002191  2.007228  2.326459 -2.075762 -1.447069 -3.178360 -1.420202   
2 -0.003956  2.366320  3.476676 -0.881520 -1.948158 -3.156266  0.327713   
3 -0.006038 -1.803261  0.401468  0.471456 -1.573445 -1.422976  2.024310   
4 -0.006830 -3.430970 -2.736555  0.581182  0.306465  0.887747  2.529270   

         P4        Cz        Pz  ...        O2        X3        X2        F7  \
0  0.099458 -0.006253  0.070874  ...  0.033154  0.004263  0.003266  0.057245   
1  4.811806 -0.334926  4.985056  ...  5.270054  0.046558  0.248616  3.810518   
2  6.043953 -0.221547  5.133423  ...  2.776656  0.014683  0.215812  7.830446   
3  2.593215 -1.560368 -1.138963  ... -7.694032  0.017682  0.073591  6.909118   
4  1.121455 -1.738621 -3.580189  ... -8.732132  0.098562  0.023940  2.107041   

         F8        X1        A2        T6        T4           TRG  


In [8]:
# Assuming we have a 'TRG' column in the data that tells us when a specific letter flickered
TRG_events = eeg_data[' TRG'] # Added a leading space in the column name

# Create segments of EEG data corresponding to each stimulus (here we assume 1 second per trial)
# Let's say we divide the data into 1-second chunks for each stimulus event
eeg_segments = []
segment_length = fs  # 1 second of data

for i, stimulus in enumerate(TRG_events.unique()):
    # Extract 1-second EEG data segments for each stimulus
    segment_start = i * segment_length
    segment_end = segment_start + segment_length
    eeg_segment = filtered_eeg.iloc[segment_start:segment_end, :]
    eeg_segments.append(eeg_segment)

# Now eeg_segments will contain 1-second long EEG data chunks for each stimulus

In [9]:
import numpy as np

# Define function to generate sine and cosine reference signals
def generate_reference_signals(freqs, time_points, fs):
    reference_signals = []
    for freq in freqs:
        t = np.arange(0, time_points) / fs
        reference_signals.append(np.sin(2 * np.pi * freq * t))
        reference_signals.append(np.cos(2 * np.pi * freq * t))
    return np.array(reference_signals)

# Assume these are the frequencies used for flickering (in Hz)
frequencies = [8, 8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4]

# Generate reference signals for a 1-second duration
time_points = segment_length
reference_signals = generate_reference_signals(frequencies, time_points, fs)

In [14]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.0 MB 2.4 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.0 MB 2.0 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.0 MB 2.0 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/11.0 MB 1.3 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.0 MB 1.1 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.0 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.0 MB 1.2 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.0 MB 1.3 MB/s eta 0:00:07
   ---------- ----------------------------- 2.9/11.0 MB 1.4 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/11.0 MB 1.4 MB/s eta 0:00:06
   ----------

In [15]:
from sklearn.cross_decomposition import CCA

# Function to apply CCA and find the best matching frequency
def apply_cca(eeg_data_segment, reference_signals):
    cca = CCA(n_components=2) # Changed n_components to 2
    correlations = []
    for i in range(0, reference_signals.shape[0], 2):
        reference_pair = reference_signals[i:i+2, :].T
        # eeg_data_segment = eeg_data_segment.values # This line is no longer needed since eeg_data_segment is already a NumPy array
        cca.fit(eeg_data_segment, reference_pair)
        _, y = cca.transform(eeg_data_segment, reference_pair)
        correlation = np.corrcoef(y[:, 0], y[:, 1])[0, 1]
        correlations.append(correlation)
    return np.argmax(correlations)

# Apply CCA to each EEG segment to detect the stimulus frequency
detected_frequencies = []
for eeg_segment in eeg_segments:
    best_match = apply_cca(eeg_segment, reference_signals)
    detected_frequencies.append(frequencies[best_match])

print(detected_frequencies)  # This will output the detected flicker frequencies

[8.8, 8.8, 9.4, 8.2]


In [16]:
# Define a mapping between frequencies and letters
frequency_to_letter = {
    8.0: 'A', 8.2: 'B', 8.4: 'C', 8.6: 'D', 8.8: 'E', 9.0: 'F', 9.2: 'G', 9.4: 'H'
}

# Convert detected frequencies to letters
detected_letters = [frequency_to_letter[freq] for freq in detected_frequencies]

print("Detected Letters:", detected_letters)

Detected Letters: ['E', 'E', 'H', 'B']


In [17]:
# Output the detected letters
print("The system detected the following sequence of letters:")
print(" ".join(detected_letters))


The system detected the following sequence of letters:
E E H B
